In [1]:
import glob

import pandas as pd

bench_dir = "/data/PycharmProjects/cytof_benchmark/results/latent_data/"
latent_files = glob.glob(bench_dir + "*/*/*/*.csv")
latent_files

['/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/CafDataset/test.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/CafDataset/val.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/CafDataset/train.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/ChallengeDataset/test.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/ChallengeDataset/val.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/ChallengeDataset/train.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/OrganoidDataset/test.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/OrganoidDataset/val.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim5/HyperSphericalVAE/OrganoidDataset/train.csv',
 '/data/PycharmProject

In [2]:
import pandas as pd

latent_list = list()
for latent_file in latent_files:
    split = latent_file.split('/')[-1].removesuffix('.csv')
    dataset_name = latent_file.split('/')[-2]
    model_name = latent_file.split('/')[-3]
    dim = latent_file.split('/')[-4]
    latent_list.append((dim,model_name,dataset_name,split,latent_file))

latent_data =  pd.pivot(
    pd.DataFrame(latent_list,columns=['dim','model','dataset','split','file']),
    index=['dim','model','dataset'],
    columns='split',
    values='file'
).reset_index()
latent_data

split,dim,model,dataset,test,train,val
0,dim2,BetaVAE,CafDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
1,dim2,BetaVAE,ChallengeDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
2,dim2,BetaVAE,OrganoidDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
3,dim2,DBetaVAE,CafDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
4,dim2,DBetaVAE,ChallengeDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
5,dim2,DBetaVAE,OrganoidDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
6,dim2,HyperSphericalVAE,CafDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
7,dim2,HyperSphericalVAE,ChallengeDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
8,dim2,HyperSphericalVAE,OrganoidDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...
9,dim2,WAE_MMD,CafDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...


In [3]:
list(list(latent_data.iterrows())[0][1])

['dim2',
 'BetaVAE',
 'CafDataset',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim2/BetaVAE/CafDataset/test.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim2/BetaVAE/CafDataset/train.csv',
 '/data/PycharmProjects/cytof_benchmark/results/latent_data/dim2/BetaVAE/CafDataset/val.csv']

In [10]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

In [8]:
dataset_dirs = {
    'OrganoidDataset':'/data/PycharmProjects/cytof_benchmark/data/organoids',
    'CafDataset':'/data/PycharmProjects/cytof_benchmark/data/caf',
    'ChallengeDataset':'/data/PycharmProjects/cytof_benchmark/data/breast_cancer_challenge',
}

variables = {
    'OrganoidDataset':['cell_type','day'],
    'CafDataset':['Patient','Culture','Treatment','Cell_type','Batch'],
    'ChallengeDataset':['treatment','cell_line'],
}

In [29]:
latent_data.iloc[20:21,:]

split,dim,model,dataset,test,train,val
20,dim3,HyperSphericalVAE,OrganoidDataset,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...,/data/PycharmProjects/cytof_benchmark/results/...


In [57]:
from pathlib import Path
import pandas as pd
import numpy as np
import datasets
import lightgbm
import sklearn

result_list = []
metadata_dict = dict()
save_dir = Path('/data/PycharmProjects/cytof_benchmark/results/classifier_data')
for row in latent_data.iterrows():
    dim,model_name,dataset_name,test_file,train_file,val_file = list(row[1])

    if dataset_name not in metadata_dict:
        dataset_class = getattr(datasets, dataset_name)
        dataset_dir = dataset_dirs[dataset_name]

        dataset = dataset_class(data_dir=dataset_dir)
        metadata_dict[dataset_name] = dataset.train[1],dataset.val[1],dataset.test[1]

    y_train,y_val,y_test = metadata_dict[dataset_name]

    for variable in variables[dataset_name]:
        gbm = lightgbm.LGBMClassifier()
        params = {'objective': 'multiclass','num_class': y_train[variable].nunique()}

        X_train = pd.read_csv(train_file,index_col=0)
        X_val = pd.read_csv(val_file,index_col=0)
        X_test = pd.read_csv(test_file,index_col=0)

        lgb_train = lightgbm.Dataset(X_train, y_train[variable].astype('category').cat.codes)
        lgb_eval = lightgbm.Dataset(X_val, y_val[variable].astype('category').cat.codes, reference=lgb_train)

        classifier = lightgbm.train(params,
                                    train_set=lgb_train,
                                    valid_sets=lgb_eval)

        y_pred = classifier.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        acc = sklearn.metrics.accuracy_score(y_test[variable].astype('category').cat.codes, y_pred)

        result_list.append((dim,model_name,dataset_name,variable,acc))

0it [00:00, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017394 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 17329221, number of used features: 2
[LightGBM] [Info] Start training from score -1.853404
[LightGBM] [Info] Start training from score -2.123202
[LightGBM] [Info] Start training from score -2.391862
[LightGBM] [Info] Start training from score -2.719639
[LightGBM] [Info] Start training from score -2.078311
[LightGBM] [Info] Start training from score -2.161447
[LightGBM] [Info] Start training from score -2.497610
[LightGBM] [Info] Start training from score -2.232530
[LightGBM] [Info] Start training from score -2.689447
[LightGBM] [Info] Start training from score -2.680407
[1]	valid_0's multi_logloss: 1.90161
[2]	valid_0's multi_logloss: 1.72186
[3]	valid_0's multi_logloss: 

1it [04:36, 276.14s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007582 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7715638, number of used features: 2
[LightGBM] [Info] Start training from score -1.249550
[LightGBM] [Info] Start training from score -3.165534
[LightGBM] [Info] Start training from score -1.726403
[LightGBM] [Info] Start training from score -1.902421
[LightGBM] [Info] Start training from score -1.667720
[LightGBM] [Info] Start training from score -1.861961
[1]	valid_0's multi_logloss: 1.6754
[2]	valid_0's multi_logloss: 1.66695
[3]	valid_0's multi_logloss: 1.66015
[4]	valid_0's multi_logloss: 1.65463
[5]	valid_0's multi_logloss: 1.65002
[6]	valid_0's multi_logloss: 1.64624
[7]	valid_0's multi_logloss: 1.64302
[8]	valid_0's multi_logloss: 1.64037
[9]	valid_0's multi_loglo

2it [07:03, 200.21s/it]

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011742 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 7715638, number of used features: 5
[LightGBM] [Info] Start training from score -1.249550
[LightGBM] [Info] Start training from score -3.165534
[LightGBM] [Info] Start training from score -1.726403
[LightGBM] [Info] Start training from score -1.902421
[LightGBM] [Info] Start training from score -1.667720
[LightGBM] [Info] Start training from score -1.861961
[1]	valid_0's multi_logloss: 1.67549
[2]	valid_0's multi_logloss: 1.66659
[3]	valid_0's multi_logloss: 1.65932
[4]	valid_0's multi_logloss: 1.65293
[5]	valid_0's multi_logloss: 1.64752
[6]	valid_0's multi_logloss: 1.64274
[7]	valid_0's multi_logloss: 1.63842
[8]	valid_0's multi_logloss: 1.63469
[9]	valid_0's multi_log

35it [1:12:33, 138.66s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001291 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 820733, number of used features: 5
[LightGBM] [Info] Start training from score -0.683185
[LightGBM] [Info] Start training from score -2.382997
[LightGBM] [Info] Start training from score -2.734975
[LightGBM] [Info] Start training from score -3.826068
[LightGBM] [Info] Start training from score -1.583719
[LightGBM] [Info] Start training from score -2.199855
[1]	valid_0's multi_logloss: 1.1771
[2]	valid_0's multi_logloss: 1.06967
[3]	valid_0's multi_logloss: 0.986893
[4]	valid_0's multi_logloss: 0.921675
[5]	valid_0's multi_logloss: 0.867963
[6]	valid_0's multi_logloss: 0.823736
[7]	valid_0's multi_logloss: 0.786229
[8]	valid_0's multi_logloss: 0.754446
[9]	valid_0's multi

36it [1:12:42, 121.18s/it]

[98]	valid_0's multi_logloss: 1.71343
[99]	valid_0's multi_logloss: 1.7133
[100]	valid_0's multi_logloss: 1.71321


In [59]:
 pd.DataFrame(result_list, columns=['dim','model','dataset','variable','test_acc']).to_csv(save_dir / "lgbm_latent.csv")

In [60]:
from sklearn.decomposition import PCA

pca_result_list = []
baseline_result_list = []
metadata_dict = dict()
save_dir = Path('/data/PycharmProjects/cytof_benchmark/results/classifier_data')
for dataset_name in variables.keys():
    dataset_class = getattr(datasets, dataset_name)
    dataset_dir = dataset_dirs[dataset_name]

    dataset = dataset_class(data_dir=dataset_dir)
    X_train,y_train = dataset.train
    X_val,y_val = dataset.train
    X_test,y_test = dataset.train

    for variable in variables[dataset_name]:
        baseline_top_class = y_test[variable].value_counts(normalize=True).max()

        gbm = lightgbm.LGBMClassifier()
        params = {'objective': 'multiclass','num_class': y_train[variable].nunique()}

        lgb_train = lightgbm.Dataset(X_train, y_train[variable].astype('category').cat.codes)
        lgb_eval = lightgbm.Dataset(X_val, y_val[variable].astype('category').cat.codes, reference=lgb_train)

        classifier = lightgbm.train(params, train_set=lgb_train, valid_sets=lgb_eval)

        y_pred = classifier.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        all_marker_acc = sklearn.metrics.accuracy_score(y_test[variable].astype('category').cat.codes, y_pred)
        baseline_result_list.append((dataset_name,variable,baseline_top_class,all_marker_acc))

    for dim in [2,3,5]:
        pca = PCA(n_components=dim)
        pca.fit(X_train)
        lgb_X_train = pca.transform(X_train)
        lgb_X_val = pca.transform(X_val)
        lgb_X_test = pca.transform(X_test)
        for variable in variables[dataset_name]:
            gbm = lightgbm.LGBMClassifier()
            params = {'objective': 'multiclass','num_class': y_train[variable].nunique()}

            lgb_train = lightgbm.Dataset(lgb_X_train, y_train[variable].astype('category').cat.codes)
            lgb_eval = lightgbm.Dataset(lgb_X_val, y_val[variable].astype('category').cat.codes, reference=lgb_train)

            classifier = lightgbm.train(params, train_set=lgb_train, valid_sets=lgb_eval)

            y_pred = classifier.predict(lgb_X_test)
            y_pred = np.argmax(y_pred, axis=1)
            acc = sklearn.metrics.accuracy_score(y_test[variable].astype('category').cat.codes, y_pred)

            pca_result_list.append((dim,dataset_name,variable,acc))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016870 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10455
[LightGBM] [Info] Number of data points in the train set: 820733, number of used features: 41
[LightGBM] [Info] Start training from score -0.683185
[LightGBM] [Info] Start training from score -2.382997
[LightGBM] [Info] Start training from score -2.734975
[LightGBM] [Info] Start training from score -3.826068
[LightGBM] [Info] Start training from score -1.583719
[LightGBM] [Info] Start training from score -2.199855
[1]	valid_0's multi_logloss: 1.087
[2]	valid_0's multi_logloss: 0.93628
[3]	valid_0's multi_logloss: 0.824137
[4]	valid_0's multi_logloss: 0.735327
[5]	valid_0's multi_logloss: 0.664285
[6]	valid_0's multi_logloss: 0.606386
[7]	valid_0's multi_logloss: 0.558273
[8]	valid_0's multi_logloss: 0.518046
[9]	valid_0's multi_logloss: 0.483504
[10]	valid_0's multi_logloss: 0.454185
[11]	

In [61]:
pd.DataFrame(pca_result_list, columns=['dim','dataset','variable','test_acc']).to_csv(save_dir / "lgbm_pca.csv")

In [62]:
pd.DataFrame(baseline_result_list, columns=['dataset','variable','maj_class_acc','lgbm_all_vars_test_acc']).to_csv(save_dir / "lgbm_baseline.csv")